# Predicting Doctor Appointment No-Shows
***

## Table of Contents
* [Introduction](#introduction)
* [Data Wrangling](#data_wrangling)
    * [General Properties](#general_properties)
    * [Data Cleaning](#data_cleaning)
        * [Checking for Duplicates](#cleaning_duplicates)
        * [Appointment ID](#cleaning_appointment_id)
        * [Scheduled Day](#cleaning_scheduled_day)
        * [Appointment Day](#cleaning_appointment_day)
        * [Gender](#cleaning_gender)
        * [Age](#cleaning_age)
        * [Bolsa Familia](#cleaning_bolsa_familia)
        * [Hypertension](#cleaning_hypertension)
        * [Diabetes](#cleaning_diabetes)
        * [Number of Handicaps](#cleaning_number_handicaps)
        * [SMS Received](#cleaning_sms_received)
        * [No Show](#cleaning_no_show)
        * [Neighborhood](#cleaning_neighborhood)
        * [Patient ID](#cleaning_patient_id)
        
    

<a id='introduction'></a>

## Introduction



In this project, my aim is to determine what patient characteristics are connected with not showing up to a doctor's appointment by analyzing a data set of over 100K medical appointments in Vitória, Espírito Santo, Brazil provided by JoniHoppen on [Kaggle](https://www.kaggle.com/joniarroba/noshowappointments).

<a id='data_wrangling'></a>

## Data Wrangling
***

In [105]:
import pandas as pd
import numpy as np
import geocoder
import config
API_KEY = config.api_key

<a id='general_properties'></a>

### General Properties

In [2]:
df = pd.read_csv('KaggleV2-May-2016.csv')
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [3]:
# Number of records
n = len(df)
n

110527

In [4]:
# Rename fields to have more consistent formatting and to English where appropriate
df.columns = ['PatientID', 'AppointmentID', 'Gender', 'ScheduledDay',
              'AppointmentDay', 'Age', 'Neighborhood', 'BolsaFamilia', \
              'Hypertension', 'Diabetes', 'Alcoholism', 'NumHandicaps', \
              'SMSReceived', 'NoShow']

In [5]:
# Number of nulls
df.isnull().sum()

PatientID         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighborhood      0
BolsaFamilia      0
Hypertension      0
Diabetes          0
Alcoholism        0
NumHandicaps      0
SMSReceived       0
NoShow            0
dtype: int64

> There are no null records in any of the fields

In [6]:
# Data types
df.dtypes

PatientID         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighborhood       object
BolsaFamilia        int64
Hypertension        int64
Diabetes            int64
Alcoholism          int64
NumHandicaps        int64
SMSReceived         int64
NoShow             object
dtype: object

<a id='data_cleaning'></a>

### Data Cleaning

<a id='cleaning_duplicates'></a>

#### Checking for Duplicates

In [30]:
df.duplicated(keep=False).any()

False

In [31]:
df.duplicated('AppointmentID', keep=False).any()

False

> There are no duplicate entries and no duplicate AppointmentIDs.

<a id='cleaning_appointment_id'></a>

#### Appointment ID

In [ ]:
# All of the appointment IDs are 7 digits long, and are between these values
min_apptID = df['AppointmentID'].min()
max_apptID = df['AppointmentID'].max()
print('{} - {}'.format(min_apptID, max_apptID))

In [ ]:
# AppointmentIDs are identifiers, so they are also converted to strings
df['AppointmentID'] = df['AppointmentID'].astype(str)

<a id='cleaning_scheduled_day'></a>

#### Scheduled Day
ScheduledDay represents the day that a patient created the appointment.

In [7]:
# Convert the scheduled day to a datetime
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])

# Verify that ScheduledDay is in a datetime format
df['ScheduledDay'].dtype

dtype('<M8[ns]')

In [8]:
# Check that the <M8[ns] is a datetime format
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')

True

In [9]:
# Verify that all records were converted to valid datetimes
np.isnat(df['ScheduledDay']).sum()

0

In [10]:
# The first date and time an appointment was scheduled 
np.min(df['ScheduledDay'])

Timestamp('2015-11-10 07:13:56')

In [11]:
# The last date and time an appointment was scheduled 
np.max(df['ScheduledDay'])

Timestamp('2016-06-08 20:07:23')

In [12]:
np.max(df['ScheduledDay']) - np.min(df['ScheduledDay'])

Timedelta('211 days 12:53:27')

> All appointments are created within an about 7-month time frame.

<a id='cleaning_appointment_day'></a>

#### Appointment Day
AppointmentDay is the day of the doctor's appointment.

In [13]:
# Convert the appointment day to a datetime
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

# Verify that AppointmentDay is in a datetime format
df['AppointmentDay'].dtype

dtype('<M8[ns]')

In [14]:
# Verify that all records were converted to valid datetimes
np.isnat(df['AppointmentDay']).sum()

0

In [15]:
# The first scheduled appointment
np.min(df['AppointmentDay'])

Timestamp('2016-04-29 00:00:00')

In [16]:
# The last scheduled appointment
np.max(df['AppointmentDay'])

Timestamp('2016-06-08 00:00:00')

In [17]:
np.max(df['AppointmentDay']) - np.min(df['AppointmentDay'])

Timedelta('40 days 00:00:00')

> While the appointments were scheduled during a 7-month period, they all were scheduled for a 40-day period.

<a id='cleaning_gender'></a>

#### Gender

In [18]:
# Gender data is clean
df['Gender'].value_counts()

F    71840
M    38687
Name: Gender, dtype: int64

<a id='cleaning_age'></a>

#### Age

In [19]:
age_counts = df['Age'].value_counts()
age_counts.index.sort_values()

Int64Index([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,
            ...
             93,  94,  95,  96,  97,  98,  99, 100, 102, 115],
           dtype='int64', length=104)

In [39]:
# Drop the record with an age of -1
invalid_age = df[df['Age'] == -1]['PatientID']
df.drop(invalid_age.index, inplace=True)

Int64Index([99832], dtype='int64')

> I removed the record with the age of -1.  All of the other ages are plausible; although the age of 115 is improbable, it is still possible.

<a id='cleaning_bolsa_familia'></a>

#### Bolsa Família

In [47]:
# BolsaFamilia data is clean
df['BolsaFamilia'].value_counts()

0    99665
1    10861
Name: BolsaFamilia, dtype: int64

<a id='cleaning_hypertension'></a>

#### Hypertension

In [48]:
# Hypertension data is clean
df['Hypertension'].value_counts()

0    88725
1    21801
Name: Hypertension, dtype: int64

<a id='cleaning_diabetes'></a>

#### Diabetes

In [49]:
# Diabetes data is clean
df['Diabetes'].value_counts()

0    102583
1      7943
Name: Diabetes, dtype: int64

<a id='cleaning_alcoholism'></a>

#### Alcholism

In [50]:
# Alcoholism data is clean
df['Alcoholism'].value_counts()

0    107166
1      3360
Name: Alcoholism, dtype: int64

<a id='cleaning_number_handicaps'></a>

#### Number of Handicaps

In [51]:
# This represents the number of handicaps a person has (as defined by the publisher of the dataset)
# All people having between 0-4 handicaps seems reasonable
df['NumHandicaps'].value_counts()

0    108285
1      2042
2       183
3        13
4         3
Name: NumHandicaps, dtype: int64

<a id='cleaning_sms_received'></a>

#### SMS Received

In [52]:
# SMS Received data is clean
df['SMSReceived'].value_counts()

0    75044
1    35482
Name: SMSReceived, dtype: int64

<a id='cleaning_no_show'></a>

#### No Show

A value of yes for NoShow means that the patient did not show up to their appointment (they were a 'no show').

In [53]:
df['NoShow'].value_counts()

No     88207
Yes    22319
Name: NoShow, dtype: int64

In [ ]:
# Convert NoShow to zeros and ones
df['NoShow'] = np.where(df['NoShow'].values == 'Yes', 1, 0)
df['NoShow'].value_counts()

<a id='cleaning_neighborhood'></a>

#### Neighborhood

The Neighborhood field has the neighborhood of the doctor's facility.

In [ ]:
neighborhood_counts = df['Neighborhood'].value_counts()
neighborhood_counts[:11]

In [ ]:
# Number of unique neighborhoods
df['Neighborhood'].nunique()

In [ ]:
# Organize the neighborhoods into a dataframe
geo = pd.DataFrame(neighborhood_counts.index, columns = ['neighborhood'])
geo.head()

API KEY

In [ ]:
# Geocode the neighborhood data
for i, row in geo.iterrows():
    # Use Bing
    full_neighborhood = row.str.title() + ', Vitória, Espírito Santo, Brazil'
    result = geocoder.bing(full_neighborhood, key=API_KEY)
    geo.at[i, 'status'] = result.status
    #geo.set_value(i, 'lat', result.lat)

In [ ]:
# Geocode the neighborhood data
for i, row in geo.iterrows():
    # Use Bing
    full_neighborhood = row.str.title() + ', Vitória, Espírito Santo, Brazil'
    result = geocoder.bing(full_neighborhood, key=API_KEY)
    geo.set_value(i, 'status', result.status) 
    geo.set_value(i, 'lat', result.lat)
    geo.set_value(i, 'lng', result.lng)
    geo.set_value(i, 'bing_neighborhood', result.neighborhood)    
    geo.set_value(i, 'city', result.city)
    geo.set_value(i, 'state', result.state)
    geo.set_value(i, 'country', result.country)
    
    # If Bing returns None for neighborhood, use Google
    if geo.loc[i, 'bing_neighborhood'] is None:
        result = geocoder.google(full_neighborhood)
        geo.set_value(i, 'status', result.status) 
        geo.set_value(i, 'lat', result.lat)
        geo.set_value(i, 'lng', result.lng)
        if result.county == 'Vitória': # Google uses county when Bing uses city
            geo.set_value(i, 'city', result.county)
        geo.set_value(i, 'state', result.state)
        if result.country == 'BR':
            geo.set_value(i, 'country', 'Brazil')

In [ ]:
geo.head()

In [ ]:
# Check that all the neighborhoods are unique
(geo['bing_neighborhood'].value_counts() == 1).all()

In [ ]:
# Check that each neighborhood returns a unique location
not any(geo.duplicated(['lat', 'lng']))

In [ ]:
# Verify that all of the locations are in Vitória, Espírito Santo, Brazil
(geo['status'] == 'OK').all()

In [ ]:
(geo['city'] == 'Vitória').all()

In [ ]:
(geo['state'] == 'ES').all()

In [ ]:
(geo['country'] == 'Brazil').all()

<a id='cleaning_patient_id'></a>

#### Patient ID

In [54]:
# Convert patient ID to a string, as it is meant to be an identifier not a number
df['PatientID'] = df['PatientID'].astype(int).astype(str)

# All identifiers are 7-15 digits long, most have at least 10
lens = df['PatientID'].apply(len)
pd.value_counts(lens)

14    39372
13    28319
15    24918
12    12835
11     4002
10      920
9       136
8        18
5         3
6         2
7         1
Name: PatientID, dtype: int64

> Most of the patient IDs are at least 10 digits long.  There is no consistent length for this field, and it is unknown, and impossible to know, if some of these patient identifiers are flawed.  It is an identifier and does not need to be operated on or with, therefore none of these records will be removed.

#### Checking PatientID Consistency

In [65]:
# Number of appointments per PatientID
appts_by_patient = df.groupby('PatientID')['AppointmentID'].count()
appts_by_patient.head()

PatientID
11111462625267     1
111124532532143    2
11114485119737     1
11116239871275     1
1111633122891      3
Name: AppointmentID, dtype: int64

There are multiple appointments for some patients, so I am checking that the information for each patient across the different appointments is the same.

In [66]:
# All have the same gender
unique_gender_vals = df.groupby('PatientID')['Gender'].nunique()
unique_gender_vals[unique_gender_vals > 1]

Series([], Name: Gender, dtype: int64)

In [73]:
# All have the same BolsaFamilia status
unique_bf_vals = df.groupby('PatientID')['BolsaFamilia'].nunique()
unique_bf_vals[unique_bf_vals > 1]

Series([], Name: BolsaFamilia, dtype: int64)

In [68]:
# All have the same value for Hypertension
unique_hypertension_vals = df.groupby('PatientID')['Hypertension'].nunique()
unique_hypertension_vals[unique_hypertension_vals > 1]

Series([], Name: Hypertension, dtype: int64)

In [74]:
# All have the same value for Diabetes
unique_diabetes_vals = df.groupby('PatientID')['Diabetes'].nunique()
unique_diabetes_vals[unique_diabetes_vals > 1]

Series([], Name: Diabetes, dtype: int64)

In [70]:
# All have the same value for Alcoholism
unique_alcoholism_vals = df.groupby('PatientID')['Alcoholism'].nunique()
unique_alcoholism_vals[unique_alcoholism_vals > 1]

Series([], Name: Alcoholism, dtype: int64)

In [72]:
# All have the same value for NumHandicaps
unique_handicaps_vals = df.groupby('PatientID')['NumHandicaps'].nunique()
unique_handicaps_vals[unique_handicaps_vals > 1]

Series([], Name: NumHandicaps, dtype: int64)

In [76]:
# All have the same value for Neighborhood - so patients kept the same doctor's office/hospital
unique_neighborhood_vals = df.groupby('PatientID')['Neighborhood'].nunique()
unique_neighborhood_vals[unique_neighborhood_vals > 1]

Series([], Name: Neighborhood, dtype: int64)

In [93]:
# Appointments occur with a 7-month period, so the entries for a patient's ages should be within a year of each other
unique_age_vals = df.groupby('PatientID')['Age'].nunique()
not_unique_age_vals = unique_age_vals[unique_age_vals > 1]

# Patients with more than one appointment, all have less than 2 values for age
not_unique_age_vals[not_unique_age_vals > 2]

Series([], Name: Age, dtype: int64)

In [100]:
# Get the information for the patients with more than one age
patients_multiple_ages = list(not_unique_age_vals.index)
multiple_ages_info = df[df['PatientID'].isin(patients_multiple_ages)].sort_values('PatientID')
multiple_ages_info.head()

,PatientID,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,BolsaFamilia,Hypertension,Diabetes,Alcoholism,NumHandicaps,SMSReceived,NoShow
3850,112114682124172,5490237,F,2016-03-18 14:26:03,2016-05-02,0,RESISTÊNCIA,0,0,0,0,0,1,Yes
110232,112114682124172,5676082,F,2016-05-09 14:56:13,2016-06-08,1,RESISTÊNCIA,0,0,0,0,0,1,Yes
10283,11238367556569,5723118,F,2016-05-20 07:58:35,2016-05-20,29,ROMÃO,0,1,0,0,0,0,No
19089,11238367556569,5675794,F,2016-05-09 14:21:41,2016-05-13,28,ROMÃO,0,1,0,0,0,0,No
19091,11238367556569,5675795,F,2016-05-09 14:21:41,2016-05-13,28,ROMÃO,0,1,0,0,0,0,No


In [103]:
# Get the difference between each patient's ages
max_age = multiple_ages_info.groupby('PatientID')['Age'].max()
min_age = multiple_ages_info.groupby('PatientID')['Age'].min()
age_diff = max_age - min_age
age_diff.head()

PatientID
112114682124172    1
11238367556569     1
1124242331227      1
1126541547466      1
112777857389857    1
Name: Age, dtype: int64

In [104]:
# Each patient's ages are within one year of each other
age_diff[age_diff <= 1].all()

True